<a href="https://colab.research.google.com/github/fjadidi2001/Insurance/blob/main/improvements_in_telematics_syn_Risk_Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('telematics_syn_updated.csv')

# Separate features and target variable
X = data.drop('Risk_Category', axis=1)
y = data['Risk_Category']

# Identify columns with non-numerical data
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply label encoding to categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Store the encoders for later use if needed

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert resampled data to a DataFrame
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Risk_Category'])], axis=1)

# Check the distribution of the target variable
print(Counter(balanced_data['Risk_Category']))

# Save the balanced dataset
balanced_data.to_csv('balanced_dataset.csv', index=False)
# Count the occurrences of 1's and 0's in Risk_Category
claim_yn_counts = df['Risk_Category'].value_counts()

print(claim_yn_counts)# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import torch

# Load the dataset from Google Drive
file_path = '/content/drive/My Drive/telematics_syn.csv'
df = pd.read_csv(file_path)

In [ ]:
import pandas as pd
import numpy as np

# Create a binary Risk Category label
df['Risk_Category'] = np.where(df['NB_Claim'] == 0, 'Low Risk', 'High Risk')

# Save the updated dataset
df.to_csv('telematics_syn_updated.csv', index=False)

# Display the first few rows to verify the new column
print(df.head())

   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity11  \
0  Urban             6213.71              25  ...                    0.0   
1  Urban            12427.42              20  ...                   24.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                    0.0   
4  Urban             6213.71              65  ...                    0.0   

   Left.turn.intensity12  Right.turn.intensity08  Right.

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('telematics_syn_updated.csv')

# Separate features and target variable
X = data.drop('Risk_Category', axis=1)
y = data['Risk_Category']

# Identify columns with non-numerical data
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply label encoding to categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Store the encoders for later use if needed

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert resampled data to a DataFrame
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Risk_Category'])], axis=1)

# Check the distribution of the target variable
print(Counter(balanced_data['Risk_Category']))

# Save the balanced dataset
balanced_data.to_csv('balanced_dataset.csv', index=False)
# Count the occurrences of 1's and 0's in Risk_Category
claim_yn_counts = df['Risk_Category'].value_counts()

print(claim_yn_counts)

Counter({'High Risk': 95728, 'Low Risk': 95728})
Risk_Category
Low Risk     95728
High Risk     4272
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the balanced dataset
balanced_data = pd.read_csv('balanced_dataset.csv')

# Identify categorical and numerical columns
categorical_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region', 'Territory']
numerical_cols = [col for col in balanced_data.columns if col not in categorical_cols + ['Risk_Category']]

# Preprocess the data: one-hot encode categorical variables and normalize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Separate features and labels
X = balanced_data.drop('Risk_Category', axis=1)
y = balanced_data['Risk_Category']

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split the data into training, validation, and testing sets (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X_preprocessed, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check the shape of the splits to ensure they are correct
print(f'Training set shape: {X_train.shape}, {y_train.shape}')
print(f'Validation set shape: {X_val.shape}, {y_val.shape}')
print(f'Testing set shape: {X_test.shape}, {y_test.shape}')

# Save the preprocessed data to CSV files if needed
train_data = pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())
train_data['Risk_Category'] = y_train.values
train_data.to_csv('train_data.csv', index=False)

val_data = pd.DataFrame(X_val, columns=preprocessor.get_feature_names_out())
val_data['Risk_Category'] = y_val.values
val_data.to_csv('val_data.csv', index=False)

test_data = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())
test_data['Risk_Category'] = y_test.values
test_data.to_csv('test_data.csv', index=False)

Training set shape: (114873, 138), (114873,)
Validation set shape: (38291, 138), (38291,)
Testing set shape: (38292, 138), (38292,)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Load the dataset
df = pd.read_csv('telematics_syn_updated.csv')

# Create Risk_Category based on the conditions
df['Risk_Category'] = np.where((df['NB_Claim'] > 1) & (df['AMT_Claim'] > 1000), 1, 0)

# Identify categorical and numerical columns
categorical_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region', 'Territory']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['Risk_Category']]

# Preprocess the data: one-hot encode categorical variables and normalize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Separate features and labels
X = df.drop('Risk_Category', axis=1)
y = df['Risk_Category']

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split the data into training, validation, and testing sets (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X_preprocessed, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Initialize TabNet model
tabnet_model = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    lambda_sparse=1e-3, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="entmax",
    scheduler_params=dict(mode="min", patience=5, min_lr=1e-5, factor=0.9),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    seed=42,
    verbose=10
)

# Train the model
tabnet_model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# Evaluate the model on the validation set
y_val_pred = tabnet_model.predict(X_val)

# Calculate accuracy and F1 score
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred, average='weighted')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")

# Save the model
saving_path_name = "tabnet_model"
saved_filepath = tabnet_model.save_model(saving_path_name)
print(f"Model saved to: {saved_filepath}")


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.05836 | train_accuracy: 0.99812 | valid_accuracy: 0.99815 |  0:00:05s
epoch 10 | loss: 0.00302 | train_accuracy: 0.99993 | valid_accuracy: 0.9997  |  0:01:06s

Early stopping occurred at epoch 18 with best_epoch = 8 and best_valid_accuracy = 0.9998


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Validation Accuracy: 0.9998
Validation F1 Score: 0.9998
Successfully saved model at tabnet_model.zip
Model saved to: tabnet_model.zip


# Update NOV 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/telematics_syn.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch_tabnet

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
from imblearn.over_sampling import SMOTE
import torch
import joblib
from typing import Tuple, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class InsuranceRiskClassifier:
    def __init__(self):
        self.categorical_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region', 'Territory']
        self.target_col = 'Risk_Category'
        self.preprocessor = None
        self.tabnet_model = None
        self.svm_model = None
        self.label_encoders = {}

    def load_and_preprocess_data(self, file_path: str) -> pd.DataFrame:
        """Load and preprocess the dataset."""
        logging.info("Loading and preprocessing data...")
        df = pd.read_csv(file_path)

        # Create risk category based on claims
        df[self.target_col] = np.where(
            (df['NB_Claim'] > 1) & (df['AMT_Claim'] > 1000),
            'High Risk',
            'Low Risk'
        )

        return df

    def prepare_features(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """Prepare features for modeling."""
        logging.info("Preparing features...")

        # Identify numerical columns
        numerical_cols = [col for col in df.columns
                         if col not in self.categorical_cols + [self.target_col]]

        # Create preprocessor if it doesn't exist
        if self.preprocessor is None:
            self.preprocessor = ColumnTransformer(
                transformers=[
                    ('num', StandardScaler(), numerical_cols),
                    ('cat', OneHotEncoder(drop='first'), self.categorical_cols)
                ])

        # Separate features and target
        X = df.drop(self.target_col, axis=1)
        y = df[self.target_col]

        # Apply preprocessing
        X_processed = self.preprocessor.fit_transform(X)

        return X_processed, y

    def apply_smote(self, X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Apply SMOTE to balance the dataset."""
        logging.info("Applying SMOTE for class balance...")
        smote = SMOTE(random_state=42)
        return smote.fit_resample(X, y)

    def split_data(self, X: np.ndarray, y: np.ndarray) -> Tuple:
        """Split data into train, validation, and test sets."""
        logging.info("Splitting data into train, validation, and test sets...")
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.4, random_state=42, stratify=y
        )
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
        )
        return X_train, X_val, X_test, y_train, y_val, y_test

    def train_tabnet(self, X_train: np.ndarray, y_train: np.ndarray,
                     X_val: np.ndarray, y_val: np.ndarray) -> None:
        """Train TabNet model."""
        logging.info("Training TabNet model...")
        self.tabnet_model = TabNetClassifier(
            n_d=64, n_a=64, n_steps=5,
            gamma=1.5, n_independent=2, n_shared=2,
            lambda_sparse=1e-3, optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            mask_type="entmax",
            scheduler_params=dict(mode="min", patience=5, min_lr=1e-5, factor=0.9),
            scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
            seed=42,
            verbose=10
        )

        self.tabnet_model.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_name=['train', 'valid'],
            eval_metric=['accuracy'],
            max_epochs=100,
            patience=10,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False
        )

    def train_svm(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """Train SVM model."""
        logging.info("Training SVM model...")
        self.svm_model = SVC(
            kernel='rbf',
            C=1.0,
            probability=True,
            random_state=42,
            class_weight='balanced'
        )
        self.svm_model.fit(X_train, y_train)

    def evaluate_models(self, X_val: np.ndarray, y_val: np.ndarray) -> Dict:
        """Evaluate both models on validation set."""
        logging.info("Evaluating models...")
        results = {}

        # Evaluate TabNet
        y_pred_tabnet = self.tabnet_model.predict(X_val)
        results['tabnet'] = {
            'accuracy': accuracy_score(y_val, y_pred_tabnet),
            'f1': f1_score(y_val, y_pred_tabnet, average='weighted'),
            'report': classification_report(y_val, y_pred_tabnet)
        }

        # Evaluate SVM
        y_pred_svm = self.svm_model.predict(X_val)
        results['svm'] = {
            'accuracy': accuracy_score(y_val, y_pred_svm),
            'f1': f1_score(y_val, y_pred_svm, average='weighted'),
            'report': classification_report(y_val, y_pred_svm)
        }

        return results

    def save_models(self, save_dir: str) -> None:
        """Save trained models and preprocessor."""
        logging.info("Saving models...")
        self.tabnet_model.save_model(f"{save_dir}/tabnet_model")
        joblib.dump(self.svm_model, f"{save_dir}/svm_model.joblib")
        joblib.dump(self.preprocessor, f"{save_dir}/preprocessor.joblib")

def main():
    # Initialize classifier
    classifier = InsuranceRiskClassifier()

    # Load and process data
    df = classifier.load_and_preprocess_data('/content/drive/My Drive/telematics_syn.csv')

    # Prepare features
    X, y = classifier.prepare_features(df)

    # Apply SMOTE
    X_balanced, y_balanced = classifier.apply_smote(X, y)

    # Split data
    X_train, X_val, X_test, y_train, y_val, y_test = classifier.split_data(X_balanced, y_balanced)

    # Train models
    classifier.train_tabnet(X_train, y_train, X_val, y_val)
    classifier.train_svm(X_train, y_train)

    # Evaluate models
    results = classifier.evaluate_models(X_val, y_val)

    # Print results
    for model_name, metrics in results.items():
        print(f"\n{model_name.upper()} Results:")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print("\nClassification Report:")
        print(metrics['report'])

    # Save models
    classifier.save_models('/content/drive/My Drive/models')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.10216 | train_accuracy: 0.99951 | valid_accuracy: 0.99965 |  0:00:13s
epoch 10 | loss: 0.00182 | train_accuracy: 0.99963 | valid_accuracy: 0.99982 |  0:02:16s

Early stopping occurred at epoch 17 with best_epoch = 7 and best_valid_accuracy = 0.9999


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



TABNET Results:
Accuracy: 0.9999
F1 Score: 0.9999

Classification Report:
              precision    recall  f1-score   support

   High Risk       1.00      1.00      1.00     19964
    Low Risk       1.00      1.00      1.00     19965

    accuracy                           1.00     39929
   macro avg       1.00      1.00      1.00     39929
weighted avg       1.00      1.00      1.00     39929


SVM Results:
Accuracy: 0.9999
F1 Score: 0.9999

Classification Report:
              precision    recall  f1-score   support

   High Risk       1.00      1.00      1.00     19964
    Low Risk       1.00      1.00      1.00     19965

    accuracy                           1.00     39929
   macro avg       1.00      1.00      1.00     39929
weighted avg       1.00      1.00      1.00     39929

Successfully saved model at /content/drive/My Drive/models/tabnet_model.zip


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
from imblearn.over_sampling import SMOTE
import torch
import joblib
from typing import Tuple, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the InsuranceRiskClassifier class
class InsuranceRiskClassifier:
    # [Paste the entire class definition here from the previous response]
    pass

# Create classifier instance and run the analysis
classifier = InsuranceRiskClassifier()

# Load and process data
df = classifier.load_and_preprocess_data('/content/drive/My Drive/telematics_syn.csv')

# Prepare features
X, y = classifier.prepare_features(df)

# Apply SMOTE
X_balanced, y_balanced = classifier.apply_smote(X, y)

# Split data
X_train, X_val, X_test, y_train, y_val, y_test = classifier.split_data(X_balanced, y_balanced)

# Train models
classifier.train_tabnet(X_train, y_train, X_val, y_val)
classifier.train_svm(X_train, y_train)

# Evaluate models
results = classifier.evaluate_models(X_val, y_val)

# Print results
for model_name, metrics in results.items():
    print(f"\n{model_name.upper()} Results:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print("\nClassification Report:")
    print(metrics['report'])

# Save models
classifier.save_models('/content/drive/My Drive/models')

AttributeError: 'InsuranceRiskClassifier' object has no attribute 'save_models'

In [ ]:
from insurance_risk_classifier import InsuranceRiskClassifier
classifier = InsuranceRiskClassifier()
# Follow the main() function steps

ModuleNotFoundError: No module named 'insurance_risk_classifier'